In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS ibge.silver

In [0]:
# Import das funções
from pyspark.sql.functions import col, when, trim
from pyspark.sql.types import IntegerType, DoubleType, StringType

In [0]:
#Lendo DataFrame na Silver
alfabetizacao_df = spark.table("ibge.bronze.alfabetizacao")

In [0]:
# Remover linha de cabeçalho residual
alfabetizacao_linha_df = (
    alfabetizacao_df
    .filter(col("NC") != "Nível Territorial (Código)")
)

# Tratamentos de tipos e limpeza
alfabetizacao_final_df = (
    alfabetizacao_linha_df
    
    # Renomeando colunas para padrão técnico
    .withColumnRenamed("NC", "cod_nivel_territorial")
    .withColumnRenamed("NN", "nivel_territorial")
    .withColumnRenamed("MN", "unid_medida")
    .withColumnRenamed("V", "valor")
    .withColumnRenamed("D1C", "cod_municipio")
    .withColumnRenamed("D1N", "municipio")
    .withColumnRenamed("D2N", "variavel")
    .withColumnRenamed("D3N", "ano")
    .withColumnRenamed("D4N", "sexo")
    .withColumnRenamed("D5N", "cor_raca")
    .withColumnRenamed("D6N", "idade")

    # Tratamento do campo valor
    .withColumn(
        "valor",
        when(col("valor") == "...", None)
        .otherwise(col("valor"))
        .cast(DoubleType())
    )

    # Conversões de tipo
    .withColumn("cod_nivel_territorial", col("cod_nivel_territorial").cast(IntegerType()))
    .withColumn("cod_municipio", col("cod_municipio").cast(IntegerType()))
    .withColumn("ano", col("ano").cast(IntegerType()))

    # Trim geral nas colunas string principais
    .withColumn("nivel_territorial", trim(col("nivel_territorial")))
    .withColumn("unid_medida", trim(col("unid_medida")))
    .withColumn("municipio", trim(col("municipio")))
    .withColumn("variavel", trim(col("variavel")))
    .withColumn("sexo", trim(col("sexo")))
    .withColumn("cor_raca", trim(col("cor_raca")))
    .withColumn("idade", trim(col("idade")))
)

In [0]:
# Caminho gerenciado por tabela
tabela_silver = "ibge.silver.alfabetizacao"

(
    alfabetizacao_final_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(tabela_silver)
)